In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Attribute Information:

1.date: Date in format dd/mm/yyyy

2.time: time in format hh:mm:ss

3.global_active_power: household global minute-averaged active power (in kilowatt)

4.global_reactive_power: household global minute-averaged reactive power (in kilowatt)

5.voltage: minute-averaged voltage (in volt)

6.global_intensity: household global minute-averaged current intensity (in ampere)

7.sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered).

8.sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light.

9.sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner.



## Load Dataset

In [2]:
data = pd.read_csv("../input/household-electrical-consumption/household_power_consumption.txt", sep=";", parse_dates=[['Date', 'Time']])

In [3]:
df = data.copy()
df

## Rename Columns & Handle Missing Data

In [4]:
df.rename(columns = {"Date_Time": "datetime", "Global_active_power": "gap", "Global_reactive_power": "grp", "Voltage" : "voltage",
                     "Global_intensity": "gi", "Sub_metering_1": "sm1", "Sub_metering_2" : "sm2", "Sub_metering_3" :"sm3" }, inplace = True)

In [5]:
df.info()

In [6]:
#Chech for Nan/missing values
df.isna().any()

In [7]:
#Filling missing data by imputation - Forward fill
df = df.fillna(method='ffill')
df.isnull().sum()

In [8]:
#Recheck missing values
df.isna().any()

Further examination shows there are special characters (?) representing missing values in the dataset

In [7]:
df[6835:6845]

In [8]:
#replace "?" with Nan
df = df.replace('?', np.nan)

In [ ]:
#df_new = df["grp"].fillna(value = df["grp"].mean())
#df_new.head()

In [ ]:
#df_new = df["grp"].fillna(value = df["grp"].mean())
#df_new.head()

In [9]:
df_na = df.copy()

In [ ]:
#Filling Nan by imputation - Forward fill
df = df.fillna(method='ffill')
df.isnull().sum()

In [ ]:
#Convert column types  to floats 
df.iloc[:, 1:] = df.iloc[:, 1:].astype("float")

In [ ]:
#Check data info
df.info()

Okay! Dataset is all ready

### More Preprocessing

In [ ]:
df_power = df.copy()
df_power.head()

In [ ]:
#Create Sum of Sub Metering for each hour
df_power["sum_sm"] = df_power.iloc[:, 5:].sum(axis=1)
df_power["sum_sm"] = round(df_power["sum_sm"], 1)

In [ ]:
df_power.head()

In [ ]:
#Set datetime as index
df_power.set_index("datetime", inplace = True)

In [ ]:
df_power.head()

## Visualization

In [ ]:
#Plot datetime X  average sub metering
plt.figure(figsize=(20,6))
plt.plot(df_power.index, df_power.sum_sm, '--', marker='*', )
plt.grid()
plt.title("Hourly Household Sub Metering")
plt.xlabel('DateTime')
plt.ylabel('Sub Metering')

Okay, Let's Make sense of this by Resampling the data into daily & weekly data

In [ ]:
df_power.isna().sum()

In [ ]:
#Resample to daily data
df_daily = df_power.resample("D").mean()

In [ ]:
df_daily.isna().sum()

In [ ]:
df_daily.shape

In [ ]:
#Plot Daily datetime X  average sub metering
plt.figure(figsize=(20,6))
plt.plot(df_daily.index, df_daily.sum_sm, '--', marker='*', )
plt.grid()
plt.title("Daily Household Sub Metering")
plt.xlabel('DateTime')
plt.ylabel('Sub Metering')

In [ ]:
#Resample to Weekly data
df_weekly = df.resample("W").mean()

In [ ]:
df_weekly.shape

In [ ]:
df_weekly.head()

In [ ]:
#Plot datetime X  average sub metering
plt.figure(figsize=(20,6))
plt.plot(df_weekly.index, df_weekly.sum_sm, '--', marker='*', )
plt.grid()
plt.title("Weekly Household Sub Metering")
plt.xlabel('DateTime')
plt.ylabel('Sub Metering')

Makes More sense! 

## Seasonality & Trend

In [ ]:
df_seas = pd.DataFrame(df_power["sum_sm"])
df_seas.head()

In [ ]:
import statsmodels.api as sm
from pylab import rcParams
rcParams['figure.figsize'] = 15, 8
decompose_series = sm.tsa.seasonal_decompose(x = df_seas["sum_sm"], model='additive', period = 1)
decompose_series.plot()
plt.show()

In [ ]:
#Interpolate Missing values for daily dataset
df_d = df_daily.sum_sm.interpolate()
df_d = pd.DataFrame(df_d)

In [ ]:
df_d

In [ ]:
import statsmodels.api as sm
from pylab import rcParams
rcParams['figure.figsize'] = 15, 8
decompose_series = sm.tsa.seasonal_decompose(x = df_d["sum_sm"], model='additive', period = 1)
decompose_series.plot()
plt.show()

In [ ]:
#Interpolate Missing values for weekly dataset
df_w = df_weekly.sum_sm.interpolate()
df_w = pd.DataFrame(df_w)

In [ ]:
import statsmodels.api as sm
from pylab import rcParams
rcParams['figure.figsize'] = 15, 8
decompose_series = sm.tsa.seasonal_decompose(x = df_w["sum_sm"], model='additive', period = 1)
decompose_series.plot()
plt.show()

So Yeah, It's a stationary data. No Seasonality and no definite trend 

In [ ]:
#Statistical check for seasonality & Trend Daily

from statsmodels.tsa.stattools import adfuller
adf = adfuller(df_d['sum_sm'])
print(f'ADF Statistic: {adf[0]}')
print(f'p-value: {adf[1]}')
print(f'No. of lags used: {adf[2]}')
print(f'No. of observations used : {adf[3]}')
print('Critical Values:')
for k, v in adf[4].items():
    print(f'   {k}: {v}') 

pvalue is less than 0.05, hence data is STATIONARY

In [ ]:
#Statistical check for seasonality & Trend Weekly

from statsmodels.tsa.stattools import adfuller
adf = adfuller(df_w['sum_sm'])
print(f'ADF Statistic: {adf[0]}')
print(f'p-value: {adf[1]}')
print(f'No. of lags used: {adf[2]}')
print(f'No. of observations used : {adf[3]}')
print('Critical Values:')
for k, v in adf[4].items():
    print(f'   {k}: {v}') 

pvalue is less than 0.05, hence data is STATIONARY

## Autocorrelation & Partial Functions

In [ ]:
#acf Daily
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#ACF
plot_acf(df_d, lags = range(0, 20))
plt.show()


In [ ]:
#acf Weekly
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#ACF
plot_acf(df_w, lags = range(0, 20))
plt.show()


In [ ]:
#PACF Daily
plot_pacf(df_d, lags = range(0, 20))
plt.show()

In [ ]:
#PACF Weekly
plot_pacf(df_w, lags = range(0, 20))
plt.show()

# Models 

## Autoregressive Integrated Moving Average (ARIMA)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
#Daily 

AR_model = ARIMA(df_d, order=(0,0,2))
AR_model_results = AR_model.fit()
plt.plot(df_d)
plt.plot(AR_model_results.fittedvalues, color='red')

In [ ]:
#Evaluation

#MAE Daily
expected = df_d["sum_sm"]
predictions = AR_model_results.fittedvalues

mae = mean_absolute_error(expected, predictions)
print('MAE: %f' % mae)

In [ ]:
def MAPE(expected,prediction):
    mape = np.mean(np.abs((expected - prediction)/expected))*100
    return mape

In [ ]:
#MAPE daily
MAPE(expected, predictions)

In [ ]:
# RMSE Daily
MSE = mean_squared_error(expected, predictions)

RMSE = math.sqrt(MSE)
print("Mean Square Error:\n",  MSE)

print("Root Mean Square Error:\n",  RMSE)

In [ ]:
#Weekly

AR_model = ARIMA(df_w, order=(0,0,2))
AR_model_results = AR_model.fit()
plt.plot(df_w)
plt.plot(AR_model_results.fittedvalues, color='red')

In [ ]:
#Evaluation

#MAE Weekly
expected = df_w["sum_sm"]
predictions = AR_model_results.fittedvalues

mae = mean_absolute_error(expected, predictions[:1457])
print('MAE: %f' % mae)

In [ ]:
#MAPE daily
MAPE(expected, predictions)

In [ ]:
# RMSE Daily
MSE = mean_squared_error(expected, predictions)

RMSE = math.sqrt(MSE)
print("Mean Square Error:\n",  MSE)

print("Root Mean Square Error:\n",  RMSE)

## The Prophet

In [ ]:
from fbprophet import Prophet
#rename columns as required

df_dd = df_d.reset_index()

In [ ]:
df_p.head()

In [ ]:
data = pd.DataFrame(df_p["gap"])
data.head()

In [ ]:
data.shape

In [ ]:
data = data.reset_index()

In [ ]:
data = data.rename(columns={"datetime": "ds", "gap": "y"})
data.head()

In [ ]:
train = data[:1095]
test = data[:-365]

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
#Daily


model = Prophet()
model.fit(train)

#predict for the next 10 months
future = model.make_future_dataframe(periods=30, freq='D') 
forecastd = model.predict(future) 
forecastd.head()
forecastd[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']]
#yhat is the prediction while yhat_lower and yhat_upper are the upper and lower boundaries


model.plot(forecastd)
plt.show()

In [ ]:
forecastd

In [ ]:
#MAPE Daily
expected = train["y"]
predictions = forecastd["yhat"]
MAPE(expected, predictions)

In [ ]:
#MAPE Daily
expected = test["y"]
predictions = forecasted["yhat"]
MAPE(expected, predictions)

In [ ]:
# RMSE Weekly
MSE = mean_squared_error(expected, predictions[:1092])

RMSE = math.sqrt(MSE)
print("Mean Square Error:\n",  MSE)

print("Root Mean Square Error:\n",  RMSE)

In [ ]:
future_test = model.make_future_dataframe(periods=30, freq='D') 
forecasted = model.predict(future_test) 
forecasted.head()
forecasted[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']]
#yhat is the prediction while yhat_lower and yhat_upper are the upper and lower boundaries


model.plot(forecasted)
plt.show()

In [ ]:
#Daily
df_dpro = df_dd.rename(columns={"datetime": "ds", "sum_sm": "y"})
df_dpro.head()

model = Prophet()
model.fit(df_dpro)

#predict for the next 10 months
future = model.make_future_dataframe(periods=30, freq='D') 
forecastd = model.predict(future) 
forecastd.head()
forecastd[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']]
#yhat is the prediction while yhat_lower and yhat_upper are the upper and lower boundaries


model.plot(forecastd)
plt.show()

In [ ]:
#Evaluation
def MAPE(expected,prediction):
    mape = np.mean(np.abs((expected - prediction)/expected))*100
    return mape

In [ ]:
#MAE Daily
expected = df_dpro["y"]
predictions = forecastd["yhat"][:1457]

mae = mean_absolute_error(expected, predictions)
print('MAE: %f' % mae)

In [ ]:
#MAPE Daily
MAPE(expected, predictions)

In [ ]:
# RMSE Daily
MSE = mean_squared_error(expected, predictions)

RMSE = math.sqrt(MSE)
print("Mean Square Error:\n",  MSE)

print("Root Mean Square Error:\n",  RMSE)


In [ ]:
#wEEKLY

#rename columns as required

df_ww = df_w.reset_index()
df_wpro = df_ww.rename(columns={"datetime": "ds", "sum_sm": "y"})
df_wpro.head()

model = Prophet()
model.fit(df_wpro)

#predict for the next 10 months
future = model.make_future_dataframe(periods=20, freq='W') 
forecast = model.predict(future) 
forecast.head()
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']]
#yhat is the prediction while yhat_lower and yhat_upper are the upper and lower boundaries


model.plot(forecast)
plt.show()

In [ ]:
#Evaluate Weekly

#MAE
expect = df_wpro["y"]
pred = forecast["yhat"]

mae = mean_absolute_error(expect, pred[:209])
print('MAE: %f' % mae)

In [ ]:
#Weekly MAPE
MAPE(expect, pred)

In [ ]:
# RMSE Weekly
MSE = mean_squared_error(expect, pred[:209])

RMSE = math.sqrt(MSE)
print("Mean Square Error:\n",  MSE)

print("Root Mean Square Error:\n",  RMSE)

In [ ]:
#Interpolate Missing values for daily dataset
df_p = df_daily.interpolate()
df_p = pd.DataFrame(df_p)

# Correlation

In [47]:
from scipy.stats import pearsonr

In [39]:
#Resample df_na 
df_corr = df.set_index("datetime")

In [40]:
#Convert column types  to floats 
df_corr.iloc[:, :] = df_corr.iloc[:, :].astype("float")

In [41]:
df_corr.head()

In [42]:
df_corr = df_corr.resample("D").mean()

In [43]:
df_corr.head()

In [44]:
#fill na with mean
df_corr["grp"] = df_corr["grp"].fillna(value = df_corr["grp"].mean())
df_corr.head()

In [45]:
#fill na with mean
df_corr["gap"] = df_corr["gap"].fillna(value = df_corr["gap"].mean())
df_corr.head()

In [54]:
#fill na with mean
df_corr["gi"] = df_corr["gi"].fillna(value = df_corr["gi"].mean())
df_corr.head()

In [55]:
#fill na with mean
df_corr["voltage"] = df_corr["voltage"].fillna(value = df_corr["voltage"].mean())
df_corr.head()

In [48]:
# calculate Pearson's correlation
data1 = df_corr["gap"]
data2 = df_corr["grp"]
corr, _ = pearsonr(data2, data1)
print('Pearsons correlation: %.2f' % corr)

In [56]:
data1 = df_corr["voltage"]
data2 = df_corr["gi"]
corr, _ = pearsonr(data2, data1)
print('Pearsons correlation: %.2f' % corr)